In [1]:
import optuna
import os
from pathlib import Path
import kaggle
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

SEED = 42
SPLITS = 5
VERSION = 4
OUTPUT = f'xg_boost_clf_submisson_v{VERSION}.csv'
N_TRIALS = 30

np.random.seed(SEED)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

In [5]:
partial_pipeline = make_pipeline(ColumnTransformer([('drop', 'drop',
                                  ['iv(g)', 't', 'b', 'n',
                                   'lOCode', 'v', 'branchCount',
                                   'e', 'i', 'lOComment'])],
                                remainder='passthrough'), SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), PowerTransformer())

In [6]:
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'tree_method':'gpu_hist',  # this parameter means using the GPU when training our model to speedup the training process
        'n_jobs': trial.suggest_categorical('n_jobs', [-1]),
        'random_state': trial.suggest_categorical('random_state', [SEED]),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': trial.suggest_int('n_estimators', 100, 600, 100),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    pipeline = make_pipeline(partial_pipeline, XGBClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=SPLITS, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

[I 2023-10-16 13:09:34,144] A new study created in memory with name: no-name-062b82a3-978f-4666-9222-95ca8d40ee9a
  0%|                                                                                                                                                                                                                 | 0/30 [00:00<?, ?it/s]/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be r

[I 2023-10-16 13:09:49,460] Trial 0 finished with value: 0.7856445715720518 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 0.0073988065515872455, 'alpha': 0.002483275813687532, 'colsample_bytree': 0.5, 'subsample': 0.4, 'learning_rate': 0.02, 'n_estimators': 400, 'max_depth': 11, 'min_child_weight': 86}. Best is trial 0 with value: 0.7856445715720518.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


[I 2023-10-16 13:10:03,087] Trial 1 finished with value: 0.781429113270524 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 5.435766209871165, 'alpha': 0.006899644864867249, 'colsample_bytree': 0.3, 'subsample': 0.6, 'learning_rate': 0.008, 'n_estimators': 400, 'max_depth': 15, 'min_child_weight': 88}. Best is trial 0 with value: 0.7856445715720518.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


...

[I 2023-10-16 13:15:03,967] Trial 27 finished with value: 0.7857265278136186 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 1.759955842083293, 'alpha': 0.0026714207580845403, 'colsample_bytree': 0.8, 'subsample': 0.7, 'learning_rate': 0.02, 'n_estimators': 400, 'max_depth': 7, 'min_child_weight': 18}. Best is trial 2 with value: 0.7861217186218017.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


[I 2023-10-16 13:15:18,868] Trial 28 finished with value: 0.785421659970495 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 0.5001251460120861, 'alpha': 0.006489011104608592, 'colsample_bytree': 0.5, 'subsample': 0.8, 'learning_rate': 0.02, 'n_estimators': 300, 'max_depth': 11, 'min_child_weight': 81}. Best is trial 2 with value: 0.7861217186218017.


/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


[I 2023-10-16 13:15:37,444] Trial 29 finished with value: 0.7850167051503686 and parameters: {'n_jobs': -1, 'random_state': 42, 'lambda': 0.2696290431278074, 'alpha': 0.0024868653583331652, 'colsample_bytree': 0.4, 'subsample': 1.0, 'learning_rate': 0.02, 'n_estimators': 400, 'max_depth': 15, 'min_child_weight': 91}. Best is trial 2 with value: 0.7861217186218017.


In [7]:
study.best_params, study.best_value

({'n_jobs': -1,
  'random_state': 42,
  'lambda': 0.5076643359572762,
  'alpha': 0.003170047051704783,
  'colsample_bytree': 0.9,
  'subsample': 0.4,
  'learning_rate': 0.02,
  'n_estimators': 300,
  'max_depth': 5,
  'min_child_weight': 76},
 0.7861217186218017)

In [8]:
pipeline = make_pipeline(partial_pipeline, XGBClassifier(**study.best_params))
pipeline.fit(X, y)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['iv(g)',
                                                                    't', 'b',
                                                                    'n',
                                                                    'lOCode',
                                                                    'v',
                                                                    'branchCount',
                                                                    'e', 'i',
                                                                    'lOComment'])])),
                                 ('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('powertransformer', PowerTransformer())]))...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None,
                               lambda=0.5076643359572762, learning_rate=0.02,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=76, missing=nan,
                               monotone_constraints=None, n_estimators=300,
                               n_jobs=-1, num_parallel_tree=None, ...))])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)